# Exploratery Data Analysis 
Understanding the dataset to explore how the data is present the dataset and if there is a need of creating some aggeregated tables that can help with:

1) Vendor selection for profitability
2) Product Pricing Optimization

In [1]:
import pandas as pd
import sqlite3

In [2]:
# Creating database connection
conn = sqlite3.connect('inventory.db')

In [3]:
# Checking tables present in the database
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", conn)
tables

,name
0,begin_inventory
1,end_inventory
2,purchases
3,purchase_prices
4,sales
5,vendor_invoice


In [4]:
# To find the total records from a particular table

# pd.read_sql("select count(*) from purchases", conn)

In [ ]:
for table in tables['name']:
    print('-' * 50, f'{table}', '-' * 50)
    print('Count of records:', pd.read_sql(f"select count (*) as count from {table}", conn)['count'].values[0])

    # To display a dataframe we have to write display function.
    display(pd.read_sql(f"select * from {table} limit 5", conn))

-------------------------------------------------- begin_inventory --------------------------------------------------
Count of records: 206529


,InventoryId,Store,City,Brand,Description,Size,onHand,Price,startDate
0,1_HARDERSFIELD_58,1,HARDERSFIELD,58,Gekkeikan Black & Gold Sake,750mL,8,12.99,2024-01-01
1,1_HARDERSFIELD_60,1,HARDERSFIELD,60,Canadian Club 1858 VAP,750mL,7,10.99,2024-01-01
2,1_HARDERSFIELD_62,1,HARDERSFIELD,62,Herradura Silver Tequila,750mL,6,36.99,2024-01-01
3,1_HARDERSFIELD_63,1,HARDERSFIELD,63,Herradura Reposado Tequila,750mL,3,38.99,2024-01-01
4,1_HARDERSFIELD_72,1,HARDERSFIELD,72,No. 3 London Dry Gin,750mL,6,34.99,2024-01-01


-------------------------------------------------- end_inventory --------------------------------------------------
Count of records: 224489


,InventoryId,Store,City,Brand,Description,Size,onHand,Price,endDate
0,1_HARDERSFIELD_58,1,HARDERSFIELD,58,Gekkeikan Black & Gold Sake,750mL,11,12.99,2024-12-31
1,1_HARDERSFIELD_62,1,HARDERSFIELD,62,Herradura Silver Tequila,750mL,7,36.99,2024-12-31
2,1_HARDERSFIELD_63,1,HARDERSFIELD,63,Herradura Reposado Tequila,750mL,7,38.99,2024-12-31
3,1_HARDERSFIELD_72,1,HARDERSFIELD,72,No. 3 London Dry Gin,750mL,4,34.99,2024-12-31
4,1_HARDERSFIELD_75,1,HARDERSFIELD,75,Three Olives Tomato Vodka,750mL,7,14.99,2024-12-31


-------------------------------------------------- purchases --------------------------------------------------


In [ ]:
# Select a particular vendor information from the given table

purchases = pd.read_sql_query("select * from purchases where VendorNumber == 4466", conn)
purchases

In [ ]:
# Purchase price from the table.
purchase_prices = pd.read_sql_query(''' select * from purchase_prices where VendorNumber == 4466''', conn)
purchase_prices

In [ ]:
vendor_invoice = pd.read_sql_query('''select * from vendor_invoice where VendorNumber == 4466''', conn)
vendor_invoice

In [ ]:
sales = pd.read_sql_query('''select * from sales where VendorNo == 4466''', conn)
sales

In [ ]:
# purchases.groupby([column name you want to see])[[summation column that you want to see]].sum() .sum() do the summation

purchases.groupby(['Brand', 'PurchasePrice'])[['Quantity', 'Dollars']].sum()

In [ ]:
purchase_prices

In [ ]:
# Check Purchase Order Number is unique

vendor_invoice['PONumber'].nunique()

In [ ]:
vendor_invoice.shape

In [ ]:
vendor_invoice.columns

In [ ]:
# We can conclude that Purchases + PurchasePrice == Vendor_invoice

purchases

In [ ]:
sales

In [ ]:
sales[['SalesDollars', 'SalesPrice']].nunique()

In [ ]:
sales.groupby(['Brand', 'SalesPrice'])[['SalesQuantity', 'SalesDollars']].sum()

1) The Purchase table contains actual purchase data, including date of purchase, products (brands) purchased by vendors, the amounts paid (in dollars),
   and the quantity purchased.
2) The purchase price column is derived from the purchase_price table, which provides product-wise actual and purchase price. The combination of vendor
   and brand is unique  in this table
3) The vendor_invoice table aggregate data from purchase table, summarizing quantity and dollars amounts, along with an additional column for freight.
   This table maintains uniqueness based on vendor and PO Number.
4) The sales table captures actual sales transactions, detailing the brands purchased by vendors, the quantity solds and the revenue earned.


As the data that we need for analysis is distributed in different tables, we need to create a summary table containing:
1) purchase transactions made by vendors
2) sales transaction data
3) freight costs (freight cost is delivery cost) for each data 
4) actual product prices from vendors

In [ ]:
vendor_invoice.columns

In [ ]:
freight_summary = pd.read_sql_query("""select VendorNumber, VendorName, SUM(Freight) as FreightCost from vendor_invoice
Group By VendorNumber""", conn)

In [ ]:
freight_summary

In [ ]:
purchases.columns

In [ ]:
purchase_prices.columns

In [ ]:
pd.read_sql_query("""select 
    p.VendorNumber, 
    p.VendorName, 
    p.Brand, 
    p.PurchasePrice, 
    pp.Volume, 
    pp.Price as ActualPrice,
    SUM(p.Quantity) as TotalPurchaseQuantity,
    SUM(p.Dollars) as TotalPurchaseDollars
    FROM purchases p
    JOIN purchase_prices pp
    ON p.Brand = pp.Brand
    WHERE p.PurchasePrice > 0
    GROUP BY p.VendorNumber, p.VendorName, p.Brand
    ORDER BY TotalPurchaseDollars""", conn)

In [ ]:
sales.columns

In [ ]:
pd.read_sql_query(""" SELECT
    VendorNo,
    Brand,
    SUM(SalesDollars) as TotalSalesDollars,
    SUM(SalesPrice) as TotalSalesPrice,
    SUM(SalesQuantity) as TotalSalesQuantity,
    SUM(ExciseTax) as TotalExciseTax
    FROM sales
    GROUP BY VendorNo, Brand""", conn)

In [ ]:
vendor_sales_summary = pd.read_sql_query("""WITH FreightSummary AS (
    SELECT
        VendorNumber,
        SUM(Freight) as FreightCost
    FROM vendor_invoice
    GROUP BY VendorNumber
),

PurchaseSummary AS(
    SELECT
        p.VendorNumber,
        p.VendorName,
        p.Brand,
        p.Description,
        p.PurchasePrice,
        pp.Price AS ActualPrice,
        pp.Volume,
        SUM(p.Quantity) AS TotalPurchaseQuantity,
        SUM(p.Dollars) AS TotalPurchaseDollars
    FROM purchases as p
    JOIN purchase_prices as pp
        ON p.Brand = pp.Brand
    WHERE p.PurchasePrice > 0
    GROUP BY p.VendorNumber, p.VendorName, p.Brand, p.Description, p.PurchasePrice, pp.Price, pp.Volume
),

SalesSummary AS(
    SELECT 
        VendorNo,
        Brand,
        SUM(SalesQuantity) AS TotalSalesQuantity,
        SUM(SalesDollars) AS TotalSalesDollars,
        SUM(SalesPrice) AS TotalSalesPrice,
        SUM(ExciseTax) AS TotalExciseTax
    FROM sales
    GROUP BY VendorNo, Brand
)

SELECT 
    ps.VendorNumber,
    ps.VendorName,
    ps.Brand,
    ps.Description,
    ps.PurchasePrice,
    ps.ActualPrice,
    ps.Volume,
    ps.TotalPurchaseQuantity,
    ps.TotalPurchaseDollars,
    ss.TotalSalesQuantity,
    ss.TotalSalesDollars,
    ss.TotalSalesPrice,
    ss.TotalExciseTax
FROM PurchaseSummary ps
LEFT JOIN SalesSummary ss
    ON ps.VendorNumber = ss. VendorNo
    AND ps.Brand = ss.Brand
LEFT JOIN FreightSummary fs
    ON ps.VendorName = fs.VendorNumber
ORDER BY ps.TotalPurchaseDollars DESC""", conn)

In [ ]:
vendor_sales_summary

Performance Optimization:
1) The query invloves heavy joins and aggregations on large datasets like sales and purchases.
2) Storing pre-aggregated results avoids repeated expensive computations
3) Helps in analyzing sales, purchases and pricing for different vendors and brands.
4) Future benefits of storing this data for faster Dashboarding & Reporting.
5) Insted of running expensive queries each time, dashboards can fetch data quickly from vendor_sales_summary.

In [ ]:
vendor_sales_summary.dtypes

In [ ]:
vendor_sales_summary.isnull().sum()

In [ ]:
vendor_sales_summary['VendorName'].unique()

In [ ]:
vendor_sales_summary['Description'].unique()

In [ ]:
vendor_sales_summary['Volume'] = vendor_sales_summary['Volume'].astype('float64')

In [ ]:
vendor_sales_summary.fillna(0, inplace=True)

In [ ]:
vendor_sales_summary['VendorName'] = vendor_sales_summary['VendorName'].str.strip()

In [ ]:
vendor_sales_summary.isnull().sum()

In [ ]:
vendor_sales_summary['GrossProfit'] = vendor_sales_summary['TotalSalesDollars'] - vendor_sales_summary['TotalPurchaseDollars']

In [ ]:
vendor_sales_summary['GrossProfit'].min()

In [ ]:
vendor_sales_summary['ProfitMargin'] = (vendor_sales_summary['GrossProfit'] / vendor_sales_summary['TotalSalesDollars']) * 100

In [ ]:
vendor_sales_summary['StockTurnOver'] = vendor_sales_summary['TotalSalesQuantity'] / vendor_sales_summary['TotalPurchaseQuantity']

In [ ]:
vendor_sales_summary['SalestoPurchaseRatio'] = vendor_sales_summary['TotalSalesDollars'] / vendor_sales_summary['TotalPurchaseDollars']

In [ ]:
vendor_sales_summary.columns

In [ ]:
cursor = conn.cursor()

In [ ]:
cursor.execute("""CREATE TABLE vendor_sales_summary (
    VendorNumber INT,
    VendorName VARCHAR(100),
    Brand INT,
    Description VARCHAR(100),
    PurchasePrice DECIMAL(10,2),
    ActualPrice DECIMAL(10,2),
    Volume,
    TotalPurchaseQuantity INT,
    TotalPurchaseDollars DECIMAL(15,2),
    TotalSalesQuantity INT,
    TotalSalesDollars DECIMAL(15,2),
    TotalSalesPrice DECIMAL(15,2),
    TotalExciseTax DECIMAL(15,2),
    GrossProfit DECIMAL(15,2),
    ProfitMargin DECIMAL(15,2),
    StockTurnOver DECIMAL(15,2),
    SalestoPurchaseRation DECIMAL(15,2),
    PRIMARY KEY (VendorNumber, Brand)
);
""")

In [ ]:
pd.read_sql_query("select * from Vendor_Summary", conn)

In [ ]:
vendor_sales_summary.to_sql('Vsales_Summary', conn, if_exists = 'replace', index = False)

In [ ]:
pd.read_sql_query("select * from Vendor_Summary", conn)

In [1]:
import sqlite3
import pandas as pd
import logging
import time

logging.basicConfig(
    filename = "logs/get_vendor_summary.log",
    level = logging.DEBUG,
    format = "%(asctime)s - %(levelname)s - %(message)s",
    filemode = "a"
)

def ingest_db(df, table_name, conn):
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    conn.commit()


def create_vendor_summary(conn):
    '''This function will merge the different tables to get the overall vendor summary and adding new columns in the resultant data'''
    start = time.time()
    vendor_sales_summary = pd.read_sql_query("""WITH FreightSummary AS (
        SELECT
            VendorNumber,
            SUM(Freight) as TotalFreightCost
        FROM vendor_invoice
        GROUP BY VendorNumber
    ),
    
    PurchaseSummary AS(
        SELECT
            p.VendorNumber,
            p.VendorName,
            p.Brand,
            p.Description,
            p.PurchasePrice,
            pp.Price AS ActualPrice,
            pp.Volume,
            SUM(p.Quantity) AS TotalPurchaseQuantity,
            SUM(p.Dollars) AS TotalPurchaseDollars
        FROM purchases as p
        JOIN purchase_prices as pp
            ON p.Brand = pp.Brand
        WHERE p.PurchasePrice > 0
        GROUP BY p.VendorNumber, p.VendorName, p.Brand, p.Description, p.PurchasePrice, pp.Price, pp.Volume
    ),
    
    SalesSummary AS(
        SELECT 
            VendorNo,
            Brand,
            SUM(SalesQuantity) AS TotalSalesQuantity,
            SUM(SalesDollars) AS TotalSalesDollars,
            SUM(SalesPrice) AS TotalSalesPrice,
            SUM(ExciseTax) AS TotalExciseTax
        FROM sales
        GROUP BY VendorNo, Brand
    )
    
    SELECT 
        ps.VendorNumber,
        ps.VendorName,
        ps.Brand,
        ps.Description,
        ps.PurchasePrice,
        ps.ActualPrice,
        ps.Volume,
        ps.TotalPurchaseQuantity,
        ps.TotalPurchaseDollars,
        ss.TotalSalesQuantity,
        ss.TotalSalesDollars,
        ss.TotalSalesPrice,
        ss.TotalExciseTax
    FROM PurchaseSummary ps
    LEFT JOIN SalesSummary ss
        ON ps.VendorNumber = ss. VendorNo
        AND ps.Brand = ss.Brand
    LEFT JOIN FreightSummary fs
        ON ps.VendorName = fs.VendorNumber
    ORDER BY ps.TotalPurchaseDollars DESC""", conn)

    return vendor_sales_summary


def clean_data(df):
    '''This function will clean the data'''
    start = time.time()
    # Changing datatype object to float64
    df['Volume'] = df['Volume'].astype('float64')

    # Filling the missing values with 0
    df.fillna(0, inplace = True)

    # Removing spaces from categorical columns
    df['VendorName'] = df['VendorName'].str.strip()
    df['Description'] = df['Description'].str.strip()

    # creating a new columns for better analysis
    df['GrossProfit'] = df['TotalSalesDollars'] - df['TotalPurchaseDollars']
    df['ProfitMargin'] = (df['GrossProfit'] / df['TotalSalesDollars']) * 100
    df['StockTurnOver'] = df['TotalSalesQuantity'] / df['TotalPurchaseQuantity']
    df['SalestoPurchaseRatio'] = df['TotalSalesDollars'] / df['TotalPurchaseDollars']
    end = time.time()
    total_time = (end - start) / 60
    return df

if __name__ == "__main__":
    # Creating a database connection
    conn = sqlite3.connect('inventory.db')

    logging.info('Creating Vendor Summary Table...')
    summary_df = create_vendor_summary(conn)
    logging.info(summary_df.head())

    logging.info('Cleaning Data...')
    clean_df = clean_data(summary_df)
    logging.info(clean_df.head())

    logging.info('Ingesting data...')
    ingest_db(clean_df, 'vendor_sales_summary', conn)
    logging.info('Completed')
    